# 第3章 推荐系统冷启动问题

## 冷启动问题简介

冷启动问题主要分3类：  
1. **用户冷启动：**主要解决如何给新用户做个性化推荐的问题
2. **物品冷启动：**主要解决如何将新的物品推荐给可能对它感兴趣的用户
3. **系统冷启动：**主要解决如何在一个新开发的网站上（还没有用户，也没有用户行为，只有一些物品的信息）设计个性化推荐，从而在网站刚发布时就让用户体验到个性化推荐服务

针对2个不同的冷启动问题，有如下解决方案：  
1. 提供非个性化的推荐：最简单的例子是热门排行榜，等到用户数据收集到一定的时候，再切换为个性化推荐
2. 利用用户注册时提供的年龄、性别等数据做粗粒度的个性化
3. 利用用户的社交网络帐号登录（需要用户授权），导入用户在社交网站上的好友信息，然后给用户推荐其好友喜欢的物品（UserCF算法）
4. 要求用户在登录时对一些物品进行反馈，收集用户对这些物品的兴趣信息，然后给用户推荐那些和这些物品相似的物品（ItemCF算法和LFM算法）
5. 对于新加入的物品，可以利用内容信息，将它们推荐给喜欢过和它们相似的物品的用户（ItemCF算法）
6. 在系统冷启动时，引入专家的知识，通过一定的高效方式迅速建立起物品的相关度表（基于图的推荐算法，PersonalRank算法）

## 利用用户注册信息

用户的注册信息分3种：
1. **人口统计学信息：**包括用户的年龄、性别、职业、民族、学历和居住地
2. **用户兴趣的描述：**让用户用文字描述他们的兴趣
3. **从其他网站导入的用户站外行为数据：**比如用户通过微博的帐号登录，就可以在得到用户同意的情况下获取用户在第三方平台的一些行为数据和社交网络数据。

基于注册信息的个性化推荐流程基本如下：  
1. 获取用户的注册信息
2. 根据用户的注册信息对用户分类
3. 给用户推荐他所属分类中用户喜欢的物品

&emsp;&emsp;基于用户注册信息的推荐算法其**核心问题**是计算每种特征的用户喜欢的物品。对于每种特征$f$，计算具有这种特征的用户对各个物品的喜好程度$p(f, i)$。$p(f, i)$定义为物品$i$在具有$f$的特征的用户中的热门程度：$$p(f, i)=\left|N(i) \cap U(f)\right|$$其中$N(i)$是喜欢该物品的用户集合，$U(f)$是具有特征$f$的用户集合。  
&emsp;&emsp;为解决用户发现他们不容易发现的物品，将$p(f,i)$定义为喜欢物品$i$的用户中具有特征$f$的比例：$$p(f, i)=\frac{|N(i) \cap U(f)|}{|N(i)| + \alpha}$$其中参数$\alpha$的目的是解决数据稀疏的问题。

## 选择合适的物品启动用户的兴趣

&emsp;&emsp;在新用户第一次访问推荐系统时，不立即给用户展示推荐系统，而是给用户提供一些物品，让用户反馈他们对这些物品的兴趣，然后根据用户反馈，提供个性化推荐。  

能够用来启动用户兴趣的物品具有以下特点：  
1. **比较热门：**让用户对热门的物品进行反馈
2. **具有代表性和区分性：**启动用户兴趣的物品不能是大众化或老少咸宜的，因为这样的物品对用户的兴趣没有区分新。
3. **启动物品集合需要有多样性：**在冷启动时，为了匹配多样的兴趣，需要提供具有很高覆盖率的启动物品集合，这些物品能覆盖几乎所有主流的用户兴趣。

&emsp;&emsp;首先，给定一群用户，用这群用户对物品评分的方差度量这群用户兴趣的一致程度。如果方差很小，说明这一群用户的兴趣不太一致，也就是说物品具有比较大的区分度，反之则说明，这群用户的兴趣比较一致。  
&emsp;&emsp;令$\sigma_{u \in U'}$为用户集合$U'$中所有评分的方差，通过如下方式度量一个物品的区分度$D(i)$：$$D(i)=\sigma_{u \in N^+(i)} + \sigma_{u \in N^-(i)} + \sigma_{u \in \bar{N}(i)}$$其中，$N^+(i)$是喜欢物品$i$的用户集合，$N^-(i)$是不喜欢物品$i$的用户集合，$\bar{N}(i)$是没有对物品$i$评分的用户集合，$\sigma_{u \in N^+(i)}$是喜欢物品$i$的用户对其他物品评分的方差，$\sigma_{u \in N^-(i)}$是不喜欢物品$i$的用户对其他物品评分的方差，$\sigma_{u \in \bar{N}(i)}$是没有对物品$i$评分的用户对其他物品的评分的方差。  

&emsp;&emsp;对于物品$i$，将用户分成3类——喜欢物品$i$的用户、不喜欢物品$i$的用户和不知道物品$i$的用户（即没有给$i$评分的用户）。如果这3类用户集合内的用户对其他的物品兴趣都不一致，说明物品$i$具有较高的区分度。

## 利用物品的内容信息

&emsp;&emsp;物品内容可以通过向量空间模型表示，该模型会将物品表示成一个关键词向量，对于中文，首先要对文本进行分词，将字流变成词流，然后从词流中检测出命名实体（如人名、地名、组织名等），这些实体和一些其他重要的词将组成关键词集合，最后对关键词进排名，计算每个关键词的权重，从而生成关键词向量。

将内容表示成一个关键词向量，对于物品$d$，关键词向量：$$d_i=\{(e_1,w_1),(e_2,w_2),\cdots\}$$其中，$e_i$就是关键词，$w_i$是关键词对应的权重。如果物品是文本，可以用信息检索领域TF-IDF公式计算词的权重：$$w_i=\frac{\text{TF}(e_i)}{\log \text{DF}(e_i)}$$  
在给定物品内容的关键词向量后，物品内容相似度可以通过向量之间的余弦相似度计算：$$w_{ij}=\frac{d_i \cdot d_j}{\sqrt{\|d_i\|\|d_j\|}}$$  

&emsp;&emsp;向量空间模型在内容数据丰富时，可以获得比较好的效果。以文本为例，如果是计算长文本的相似度，用向量空间末利用关键词计算相似度，已经可以获得很好的精确度。但是，如果文本很短，关键词很少，向量空间模型就很难计算出准确的相似度。

## 发挥专家的作用

&emsp;&emsp;很多推荐系统在建立时，既没有用户的行为数据，也灭有充足的物品内容信息来计算准确的物品相似度。那么，为了在推荐系统建立时，就让用户得到比较好的体验，很多系统都利用专家进行标注。  
&emsp;&emsp;Jinni在电影基因工程中采用了半人工、半自动的方式：  
    1. 让专家对电影进行标注，每个电影多有大约50个基因，这些基因来自大约1000个基因库
    2. 使用NLP和机器学习，通过分析用户对电影的评论和电影的一些内容属性，对电影（特别是新电影）进行自己的标注
    3. 设计了让用户对基因进行反馈的界面

&emsp;&emsp;Jinni通过专家和机器学习相结合的方式，解决了系统冷启动的问题。